In [1]:
import gymnasium as gym
import time
import numpy as np
import platform
import matplotlib.pyplot
import torch
import os
from importlib.metadata import version
from stable_baselines3 import PPO, DQN
from stable_baselines3.common.env_util import make_vec_env
from stable_baselines3.common.vec_env import VecFrameStack, VecVideoRecorder, VecTransposeImage
from stable_baselines3.common.callbacks import BaseCallback, EvalCallback
from stable_baselines3.common.evaluation import evaluate_policy
from stable_baselines3.common.atari_wrappers import WarpFrame
from stable_baselines3.common.env_checker import check_env
from gymnasium.envs.box2d.car_racing import CarRacing

In [2]:
class DiscreteCarRacingWrapper(gym.ActionWrapper):
    def action(self, action):
        return int(action)
env = gym.make('CarRacing-v3')
# env = DiscreteCarRacingWrapper(env)
print("Observation Space Size: ", env.observation_space)
print("Action Space Size: ", env.action_space)
env.close()

Observation Space Size:  Box(0, 255, (96, 96, 3), uint8)
Action Space Size:  Box([-1.  0.  0.], 1.0, (3,), float32)


In [3]:
env_str = "CarRacing-v3"
log_dir = "./logs/{}".format(env_str)
env_kwargs_dict={"continuous": False}
gray_scale = True

# If gray_scale True, convert obs to gray scale 84 x 84 image
wrapper_class = WarpFrame if gray_scale else None

In [ ]:
# Create Training CarRacing environment
env = make_vec_env(env_str,
                   n_envs=1,
                   env_kwargs=env_kwargs_dict,
                   wrapper_class=wrapper_class)
env = VecFrameStack(env, n_stack=4)
env = VecTransposeImage(env)

# Create Evaluation CarRacing environment
env_val = make_vec_env(env_str,
                       n_envs=1,
                       env_kwargs=env_kwargs_dict,
                       wrapper_class=wrapper_class)
env_val = VecFrameStack(env_val, n_stack=4)
env_val = VecTransposeImage(env_val)

# Create Evaluation Callback
# eval_freq - can cause learning instability if set to low
eval_callback = EvalCallback(env_val,
                             best_model_save_path=log_dir,
                             log_path=log_dir,
                             eval_freq=25_000,
                             render=False,
                             n_eval_episodes=20)

# Initialize DQN
# buffer_size - encourages exploration of other actions
model = PPO('CnnPolicy',
            env,
            verbose=1,
            )

# Train the model
model.learn(total_timesteps=750000,
            progress_bar=True,
            callback=eval_callback)

# Save the model
model.save(os.path.join(log_dir, "dqn_car_racing"))

mean_reward, std_reward = evaluate_policy(model, env, n_eval_episodes=20)
print(f"Mean reward: {mean_reward:.2f} +/- {std_reward:.2f}")

env.close()
env_val.close()
     

Using cpu device


Output()

InvalidAction: you passed the invalid action `2.0`. The supported action_space is `Discrete(5)`